In [18]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input


class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, build_fn, input_shape=None, epochs=100, batch_size=32, verbose=0):
        self.build_fn = build_fn
        self.input_shape = input_shape
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None

    def fit(self, X, y):
        if self.input_shape:
            X = np.asarray(X).reshape(-1, *self.input_shape)  # Convertir a array NumPy y luego hacer reshape
        else:
            X = np.asarray(X)
        self.model = self.build_fn(self.input_shape)
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        return self

    def predict(self, X):
        if self.input_shape:
            X = np.asarray(X).reshape(-1, *self.input_shape)  # Convertir a array NumPy y luego hacer reshape
        else:
            X = np.asarray(X)
        return self.model.predict(X).flatten()

# Función para crear un MLP (Multi-Layer Perceptron) en Keras
def create_mlp_model(input_shape):
    model = Sequential()
    model.add(Dense(100, input_dim=input_shape[0], activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

# Función para crear una RNN (Recurrent Neural Network) en Keras
def create_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(50, input_shape=input_shape, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

# Función para crear una LSTM (Long Short-Term Memory) en Keras
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, input_shape=input_shape, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

# Función para crear una GRU (Gated Recurrent Unit) en Keras
def create_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(50, input_shape=input_shape, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

# Función para crear una CNN (Convolutional Neural Network) en Keras
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=2, activation='relu', input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return model

# Definir una lista de modelos a evaluar
modelos = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression': LinearRegression(),  # Usaremos PolynomialFeatures más adelante
    'Neural Network (MLP)': KerasRegressorWrapper(build_fn=create_mlp_model, input_shape=(3,), epochs=100),
    'RNN': KerasRegressorWrapper(build_fn=create_rnn_model, input_shape=(3, 1), epochs=100),
    'LSTM': KerasRegressorWrapper(build_fn=create_lstm_model, input_shape=(3, 1), epochs=100),
    'GRU': KerasRegressorWrapper(build_fn=create_gru_model, input_shape=(3, 1), epochs=100),
    'CNN': KerasRegressorWrapper(build_fn=create_cnn_model, input_shape=(3, 1), epochs=100),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

# Continua con la implementación del código...


In [ ]:
# Cargar los datos desde los archivos proporcionados
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_minas_df = pd.read_excel('Indicadores_Minas.xlsx')
indicadores_carguio_df = pd.read_excel('Indicadores_Carguio.xlsx')
indicadores_perforacion_df = pd.read_excel('Indicadores_Perforación.xlsx')
indicadores_transporte_df = pd.read_excel('Indicadores_Transporte.xlsx')

# Convertir las fechas a formato datetime en costos
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])

# Convertir las fechas a formato datetime en los DataFrames de indicadores
for df in [indicadores_minas_df, indicadores_carguio_df, indicadores_perforacion_df, indicadores_transporte_df]:
    df['Fecha'] = pd.to_datetime(df['Fecha'])

# Unir todos los DataFrames de indicadores en uno solo
indicadores_df = indicadores_minas_df.copy()
indicadores_df = pd.merge(indicadores_df, indicadores_carguio_df, on='Fecha', how='inner', suffixes=('_minas', '_carguio'))
indicadores_df = pd.merge(indicadores_df, indicadores_perforacion_df, on='Fecha', how='inner', suffixes=('', '_perforacion'))
indicadores_df = pd.merge(indicadores_df, indicadores_transporte_df, on='Fecha', how='inner', suffixes=('', '_transporte'))

# Unir el DataFrame de costos con el DataFrame combinado de indicadores
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])  # Eliminar la columna duplicada de Fecha

# Calcular la correlación entre todos los subprocesos y todos los indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha

correlation_matrix = df_unificado.corr()
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]

# Crear una tabla con los subprocesos, indicadores más correlacionados y sus valores de correlación
resultados_lista = []
for subproceso in correlation_filtered.index:
    for indicador, correlacion in correlation_filtered.loc[subproceso].nlargest(3).items():
        resultados_lista.append({
            'Subproceso': subproceso,
            'Indicador': indicador,
            'Correlación': correlacion
        })

df_resultados = pd.DataFrame(resultados_lista)

# Guardar los resultados en un archivo Excel
df_resultados.to_excel('Mejor Indicador por Subproceso.xlsx', index=False)

# Continuar con la división de los datos para entrenamiento y prueba

# Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
test_data = df_unificado[(df_unificado['Subproceso_Costo'] >= '2022-01-01') & (df_unificado['Subproceso_Costo'] < '2023-01-01')]

# Definir una lista de modelos a evaluar
modelos = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression': LinearRegression(),  # Usaremos PolynomialFeatures más adelante
    'Neural Network (MLP)': KerasRegressorWrapper(build_fn=create_mlp_model, input_shape=(3,), epochs=100),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

# Crear un diccionario para almacenar los resultados de cada modelo
resultados_modelos = {}

# Realizar predicciones para cada modelo
for nombre_modelo, modelo in modelos.items():
    resultados = {}
    for subproceso in subprocesos_cols:
        indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
        if len(indicadores_seleccionados) > 0:
            X_train = train_data[indicadores_seleccionados]
            y_train = train_data[subproceso]
            X_test = test_data[indicadores_seleccionados]
            
            if nombre_modelo == 'Polynomial Regression':
                poly = PolynomialFeatures(degree=2)
                X_train = poly.fit_transform(X_train)
                X_test = poly.transform(X_test)
                modelo.fit(X_train, y_train)
            else:
                modelo.fit(X_train, y_train)
            
            y_pred_2022 = modelo.predict(X_test)
            costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] >= '2022-01-01'][subproceso].values
            
            # Calcular las diferencias y diferencias porcentuales
            diferencias = y_pred_2022 - costo_real_2022
            diferencias_pct = [(dif / real) * 100 if real != 0 else 'N/A' for dif, real in zip(diferencias, costo_real_2022)]
            
            # Guardar resultados en el diccionario
            for mes, pred, real, dif, dif_pct in zip(pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"), y_pred_2022, costo_real_2022, diferencias, diferencias_pct):
                resultados.setdefault(subproceso, {}).update({
                    f'Predicción {mes}': pred,
                    f'Costo Real {mes}': real,
                    f'Diferencia {mes}': dif,
                    f'Diferencia % {mes}': dif_pct
                })
    
    # Convertir los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados).T
    resultados_modelos[nombre_modelo] = resultados_df

# Comparar los resultados entre modelos para cada subproceso y mes
mejor_modelo_df = pd.DataFrame(columns=['Subproceso', 'Mes', 'Mejor Modelo', 'Diferencia %'])

for subproceso in subprocesos_cols:
    for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
        mejor_modelo = None
        menor_diferencia = np.inf
        
        for nombre_modelo, df in resultados_modelos.items():
            diferencia_col = f'Diferencia % {mes}'
            if diferencia_col in df.columns and subproceso in df.index and df.loc[subproceso, diferencia_col] != 'N/A':
                diferencia = abs(df.loc[subproceso, diferencia_col])
                if diferencia < menor_diferencia:
                    menor_diferencia = diferencia
                    mejor_modelo = nombre_modelo
        
        # Guardar el mejor modelo para el subproceso y mes
        nueva_fila = pd.DataFrame({
            'Subproceso': [subproceso],
            'Mes': [mes],
            'Mejor Modelo': [mejor_modelo],
            'Diferencia %': [menor_diferencia]
        })
        
        mejor_modelo_df = pd.concat([mejor_modelo_df, nueva_fila], ignore_index=True)

# Exportar como Excel los resultados
mejor_modelo_df.to_excel(r'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\Costos_Indicadores_Mina_Mejor_Modelo.xlsx', index=False)

# Mostrar los resultados
print(mejor_modelo_df)


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Crear DataFrames para almacenar los mejores modelos y sus diferencias
mejores_modelos_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))
diferencias_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))
# Comparar los resultados entre modelos para cada subproceso y mes
for subproceso in subprocesos_cols:
    for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
        mejor_modelo = None
        menor_diferencia = np.inf
        
        for nombre_modelo, df in resultados_modelos.items():
            diferencia_col = f'Diferencia % {mes}'
            if diferencia_col in df.columns and subproceso in df.index and df.loc[subproceso, diferencia_col] != 'N/A':
                diferencia = abs(df.loc[subproceso, diferencia_col])
                if diferencia < menor_diferencia:
                    menor_diferencia = diferencia
                    mejor_modelo = nombre_modelo
        
        # Guardar el mejor modelo y su diferencia
        mejores_modelos_df.loc[subproceso, mes] = mejor_modelo
        diferencias_df.loc[subproceso, mes] = menor_diferencia

# Exportar los resultados a un archivo Excel con dos hojas
output_file_path = r'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\ResultadoFinal.xlsx'

with pd.ExcelWriter(output_file_path) as writer:
    mejores_modelos_df.to_excel(writer, sheet_name='Mejores Modelos')
    diferencias_df.to_excel(writer, sheet_name='Diferencias %')

# Mostrar los primeros registros de cada DataFrame
mejores_modelos_df.head(), diferencias_df.head()

(                                January-2022         February-2022  \
 ADM OPERACIÓN Y GESTIÓN ESP             None     Linear Regression   
 ADM OPERACIÓN Y GESTIÓN ESP SUR      XGBoost                   RNN   
 ADM OPERACIÓN Y GESTIÓN LLANO           None                  None   
 ADM OPERACIÓN Y GESTIÓN MIR             None                  LSTM   
 ADM OPERACIÓN Y GESTIÓN OXE             None  Neural Network (MLP)   
 
                                             March-2022     April-2022  \
 ADM OPERACIÓN Y GESTIÓN ESP          Gradient Boosting  Random Forest   
 ADM OPERACIÓN Y GESTIÓN ESP SUR  Polynomial Regression            CNN   
 ADM OPERACIÓN Y GESTIÓN LLANO                     None           None   
 ADM OPERACIÓN Y GESTIÓN MIR                       LSTM  Random Forest   
 ADM OPERACIÓN Y GESTIÓN OXE                        RNN            CNN   
 
                                           May-2022 June-2022  \
 ADM OPERACIÓN Y GESTIÓN ESP      Gradient Boosting   XGBoost